In [1]:
import pandas as pd
import fsspec
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import os

def tree(fs, path, prefix=""):
    """Recursively list the contents of a directory in a tree-like format."""
    # print(path)
    items = fs.ls(path, detail=True)
    for i, item in enumerate(items):
        is_last = i == (len(items) - 1)  # Check if this is the last item
        if item['type'] == 'directory':
            # Print the directory name
            print(f"{prefix}{'└── ' if is_last else '├── '}{item['name'].split('/')[-1]}")
            # Recursively list this directory's contents
            new_prefix = prefix + ('    ' if is_last else '│   ')
            tree(fs, item['name'], new_prefix)
        else:
            # Print the file name
            print(f"{prefix}{'└── ' if is_last else '├── '}{item['name'].split('/')[-1]}")


<span style="font-size:2em; color:red;">5.1 ชื่อไฟล์</span>


In [2]:
gcs_token_path = "C:\\Users\\User\\Downloads\\teacher-dsi310-2023.json"
catalog_path = 'gcs://dsi310_bucket/group10/'
fs=fsspec.filesystem('gcs', token=gcs_token_path)
with fs.open(catalog_path+'readme.md','wb') as f:
    
    f.write(b'# Hello')
tree(fs,catalog_path)

├── Sanitary Napkins 2
│   └── date=2023-12-16
│       └── 173ad84af7b440df90cd3fd1c6341810-0.parquet
├── Sanitary Napkins
│   ├── date=2023-12-15
│   │   └── fef48a66aedf4957a766291da9bff994-0.parquet
│   └── date=2023-12-16
│       └── bebf4c49f4fc434d8775b20ad648827a-0.parquet
├── 
├── Sanitary Napkins.parquet
└── readme.md


In [3]:
xlsx_file_path = "cleaning data.xlsx"

# ใช้ pandas อ่านข้อมูลจาก Excel เป็น DataFrame
df_excel = pd.read_excel(xlsx_file_path)

# ใช้ PyArrow แปลง DataFrame เป็น Arrow Table
table_arrow = pa.Table.from_pandas(df_excel)

In [4]:
# กำหนดที่อยู่ของไฟล์ Excel
xlsx_file_path = "cleaning data.xlsx"

# ใช้ pandas อ่านข้อมูลจาก Excel เป็น DataFrame
df_xlsx = pd.read_excel(xlsx_file_path)

# แปลง 'date' เป็นวันที่
df_xlsx['date'] = pd.to_datetime('2023-12-16').date()

# แสดงผล DataFrame
print(df_xlsx)


       brand   price    sale  total_sale       location   usage  length type  \
0       Sofy   10.75    74.0       795.5        Bangkok   night    29.0  pad   
1       Sofy   33.00  3269.0    107877.0        Bangkok     day    22.0  pad   
2       Sofy   11.00   748.0      8228.0        Bangkok   night    29.0  pad   
3       Sofy   59.00   819.0     48321.0        Bangkok   night    29.0  pad   
4       Sofy   39.00   426.0     16614.0        Bangkok  normal    22.0  pad   
...      ...     ...     ...         ...            ...     ...     ...  ...   
5774  Sanita  109.00   180.0     19620.0  กรุงเทพมหานคร   night    29.0  pad   
5775  Sanita  960.00   129.0    123840.0  กรุงเทพมหานคร   night    29.0  pad   
5776  Sanita  129.00    91.0     11739.0  กรุงเทพมหานคร  normal    24.5  pad   
5777  Sanita  119.00    62.0      7378.0  กรุงเทพมหานคร  normal    24.5  pad   
5778  Sanita  129.00   164.0     21156.0  กรุงเทพมหานคร  normal    24.5  pad   

     wing   size        date  
0      n

In [5]:
# แสดงข้อมูลทั่วไปเกี่ยวกับ DataFrame
df_xlsx.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779 entries, 0 to 5778
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   brand       5779 non-null   object 
 1   price       5779 non-null   float64
 2   sale        5634 non-null   float64
 3   total_sale  5779 non-null   float64
 4   location    5779 non-null   object 
 5   usage       5779 non-null   object 
 6   length      5498 non-null   float64
 7   type        5779 non-null   object 
 8   wing        5779 non-null   object 
 9   size        5532 non-null   object 
 10  date        5779 non-null   object 
dtypes: float64(4), object(7)
memory usage: 496.8+ KB


In [6]:
# Example DataFrame
df = df_xlsx

metadata = {
    'data_type':2,
    'title': 'Sales of Sanitary Napkins on Lazada',
    'owner_org': 'บริษัท ลาซาด้า จำกัด',
    'maintainer': 'กลุ่มอาลีบาบา',
    'maintainer_email': 'sevice@lazada.co.th',
    'tag_string': 'brand, price, sale, total_sale, location, usage, length, type, wing, size',
    'note': 'ยอดขายจากการจำหน่ายผ้าอนามัย,จำแนกตามรายชื่อยี่ห้อ,รูปแบบของผ้าอนามัย',
    'objective': 99,
    'update_frequency_unit': 'R',
    'update_frequency_interval': 0,
    'geo_coverage': '04',
    'data_source': 'ข้อมูลการขายสินค้าและยอดขายในการให้บริการมีที่มาจากบริษัท ลาซาด้า จำกัด',
    'data_format': 10,
    'data_category': 1,
    'license_id': 98
}

data_dictionary = {
    'brand': 'ชื่อยี่ห้อผ้าอนามัย',
    'price': 'ราคา',
    'sale': 'จำนวนที่ขาย',
    'total_sale':'ยอดขายรวม',
    'location': 'จังหวัด',
    'usage':'การใช้งาน',
    'length':'ความยาว',
    'type':'รูปแบบ',
    'wing':'มีปีก,ไม่มีปีก',
    'size':'ขนาด',
    'date': 'วันที่ได้การนำข้อมูลมา'
}

# Convert DataFrame to PyArrow Table with metadata
table = pa.Table.from_pandas(df)
table = table.replace_schema_metadata({'metadata': str(metadata), 'dictionary': str(data_dictionary)})

table.schema
table.schema

brand: string
price: double
sale: double
total_sale: double
location: string
usage: string
length: double
type: string
wing: string
size: string
date: date32[day]
-- schema metadata --
metadata: '{'data_type': 2, 'title': 'Sales of Sanitary Napkins on Lazada' + 867
dictionary: '{'brand': 'ชื่อยี่ห้อผ้าอนา�' + 421

<span style="font-size:2em; color:red;">5.2 เมทาดาตา 14 field</span>

In [7]:
# Extracting metadata and data dictionary
metadata = eval(table.schema.metadata[b'metadata']) if b'metadata' in table.schema.metadata else None
data_dictionary = eval(table.schema.metadata[b'dictionary']) if b'dictionary' in table.schema.metadata else None

# Display the DataFrame, Metadata, and Data Dictionary
print("Metadata:", metadata)
print("Data Dictionary:", data_dictionary)

Metadata: {'data_type': 2, 'title': 'Sales of Sanitary Napkins on Lazada', 'owner_org': 'บริษัท ลาซาด้า จำกัด', 'maintainer': 'กลุ่มอาลีบาบา', 'maintainer_email': 'sevice@lazada.co.th', 'tag_string': 'brand, price, sale, total_sale, location, usage, length, type, wing, size', 'note': 'ยอดขายจากการจำหน่ายผ้าอนามัย,จำแนกตามรายชื่อยี่ห้อ,รูปแบบของผ้าอนามัย', 'objective': 99, 'update_frequency_unit': 'R', 'update_frequency_interval': 0, 'geo_coverage': '04', 'data_source': 'ข้อมูลการขายสินค้าและยอดขายในการให้บริการมีที่มาจากบริษัท ลาซาด้า จำกัด', 'data_format': 10, 'data_category': 1, 'license_id': 98}
Data Dictionary: {'brand': 'ชื่อยี่ห้อผ้าอนามัย', 'price': 'ราคา', 'sale': 'จำนวนที่ขาย', 'total_sale': 'ยอดขายรวม', 'location': 'จังหวัด', 'usage': 'การใช้งาน', 'length': 'ความยาว', 'type': 'รูปแบบ', 'wing': 'มีปีก,ไม่มีปีก', 'size': 'ขนาด', 'date': 'วันที่ได้การนำข้อมูลมา'}


<span style="font-size:2em; color:red;">5.3 เวลาในการส่งข้อมูลไปยัง GCS</span>

In [8]:
# Write to GCS
gcs_path = 'gcs://dsi310_bucket/group10/'  # Replace with your bucket path
with fs.open(path=catalog_path+'Sanitary Napkins.parquet',mode='wb') as f:  # Replace with your GCS token
    pq.write_table(table, f)

tree(fs,catalog_path)

├── Sanitary Napkins 2
│   └── date=2023-12-16
│       └── 173ad84af7b440df90cd3fd1c6341810-0.parquet
├── Sanitary Napkins
│   ├── date=2023-12-15
│   │   └── fef48a66aedf4957a766291da9bff994-0.parquet
│   └── date=2023-12-16
│       └── bebf4c49f4fc434d8775b20ad648827a-0.parquet
├── 
├── Sanitary Napkins.parquet
└── readme.md


In [9]:
# Example DataFrame
df = df_xlsx

metadata = {
    'data_type':2,
    'title': 'Sales of Sanitary Napkins on Lazada',
    'owner_org': 'บริษัท ลาซาด้า จำกัด',
    'maintainer': 'กลุ่มอาลีบาบา',
    'maintainer_email': 'sevice@lazada.co.th',
    'tag_string': 'brand, price, sale, total_sale, location, usage, length, type, wing, size',
    'note': 'ยอดขายจากการจำหน่ายผ้าอนามัย,จำแนกตามรายชื่อยี่ห้อ,รูปแบบของผ้าอนามัย',
    'objective': 99,
    'update_frequency_unit': 'R',
    'update_frequency_interval': 0,
    'geo_coverage': '04',
    'data_source': 'ข้อมูลการขายสินค้าและยอดขายในการให้บริการมีที่มาจากบริษัท ลาซาด้า จำกัด',
    'data_format': 10,
    'data_category': 1,
    'license_id': 98
}

data_dictionary = {
    'brand': 'ชื่อยี่ห้อผ้าอนามัย',
    'price': 'ราคา',
    'sale': 'จำนวนที่ขาย',
    'total_sale':'ยอดขายรวม',
    'location': 'จังหวัด',
    'usage':'การใช้งาน',
    'length':'ความยาว',
    'type':'รูปแบบ',
    'wing':'มีปีก,ไม่มีปีก',
    'size':'ขนาด',
    'date': 'วันที่ได้การนำข้อมูลมา'
}

# Convert DataFrame to PyArrow Table with metadata
table = pa.Table.from_pandas(df)
table = table.replace_schema_metadata({'metadata': str(metadata), 'dictionary': str(data_dictionary)})

table.schema
table.schema
df

,brand,price,sale,total_sale,location,usage,length,type,wing,size,date
0,Sofy,10.75,74.0,795.5,Bangkok,night,29.0,pad,no,thick,2023-12-16
1,Sofy,33.00,3269.0,107877.0,Bangkok,day,22.0,pad,yes,thin,2023-12-16
2,Sofy,11.00,748.0,8228.0,Bangkok,night,29.0,pad,no,thick,2023-12-16
3,Sofy,59.00,819.0,48321.0,Bangkok,night,29.0,pad,yes,thin,2023-12-16
4,Sofy,39.00,426.0,16614.0,Bangkok,normal,22.0,pad,no,thin,2023-12-16
...,...,...,...,...,...,...,...,...,...,...,...
5774,Sanita,109.00,180.0,19620.0,กรุงเทพมหานคร,night,29.0,pad,yes,thick,2023-12-16
5775,Sanita,960.00,129.0,123840.0,กรุงเทพมหานคร,night,29.0,pad,yes,thick,2023-12-16
5776,Sanita,129.00,91.0,11739.0,กรุงเทพมหานคร,normal,24.5,pad,yes,thick,2023-12-16
5777,Sanita,119.00,62.0,7378.0,กรุงเทพมหานคร,normal,24.5,pad,yes,thin,2023-12-16


In [10]:
# Define GCS path and write to GCS

# No need to open a file with fsspec, use the path directly
dataset_name ='Sanitary Napkins 2'
path = catalog_path+dataset_name
path

'gcs://dsi310_bucket/group10/Sanitary Napkins 2'

In [11]:
pq.write_to_dataset(table, root_path=path, partition_cols=['date'], filesystem=fs,)

tree(fs,catalog_path)

├── Sanitary Napkins 2
│   └── date=2023-12-16
│       ├── 173ad84af7b440df90cd3fd1c6341810-0.parquet
│       └── ecacd2104e5e4c718ab3404cb55b3b12-0.parquet
├── Sanitary Napkins
│   ├── date=2023-12-15
│   │   └── fef48a66aedf4957a766291da9bff994-0.parquet
│   └── date=2023-12-16
│       └── bebf4c49f4fc434d8775b20ad648827a-0.parquet
├── 
├── Sanitary Napkins.parquet
└── readme.md


In [12]:
my_path=catalog_path+'Sanitary Napkins 2'
files = ["gs://"  + path for path in fs.glob(my_path + "/*/*")]
files

['gs://dsi310_bucket/group10/Sanitary Napkins 2/date=2023-12-16/173ad84af7b440df90cd3fd1c6341810-0.parquet',
 'gs://dsi310_bucket/group10/Sanitary Napkins 2/date=2023-12-16/ecacd2104e5e4c718ab3404cb55b3b12-0.parquet']

In [13]:
import pyarrow as pa
pq.read_table(files,filesystem=fs)

pyarrow.Table
brand: string
price: double
sale: double
total_sale: double
location: string
usage: string
length: double
type: string
wing: string
size: string
date: dictionary<values=string, indices=int32, ordered=0>
----
brand: [["Sofy","Sofy","Sofy","Sofy","Sofy",...,"Sanita","Sanita","Sanita","Sanita","Sanita"],["Sofy","Sofy","Sofy","Sofy","Sofy",...,"Sanita","Sanita","Sanita","Sanita","Sanita"]]
price: [[10.75,33,11,59,39,...,109,960,129,119,129],[10.75,33,11,59,39,...,109,960,129,119,129]]
sale: [[74,3269,748,819,426,...,180,129,91,62,164],[74,3269,748,819,426,...,180,129,91,62,164]]
total_sale: [[795.5,107877,8228,48321,16614,...,19620,123840,11739,7378,21156],[795.5,107877,8228,48321,16614,...,19620,123840,11739,7378,21156]]
location: [["Bangkok","Bangkok","Bangkok","Bangkok","Bangkok",...,"กรุงเทพมหานคร","กรุงเทพมหานคร","กรุงเทพมหานคร","กรุงเทพมหานคร","กรุงเทพมหานคร"],["Bangkok","Bangkok","Bangkok","Bangkok","Bangkok",...,"กรุงเทพมหานคร","กรุงเทพมหานคร","กรุงเทพมหานคร","กรุงเทพ

In [14]:
import pyarrow.dataset as ds

# dataset = ds.dataset(files, format="parquet", filesystem=fs)
# table = dataset.to_table()

dataset = pq.ParquetDataset(path_or_paths=files, filesystem=fs)
table = dataset.read()

table.schema

brand: string
price: double
sale: double
total_sale: double
location: string
usage: string
length: double
type: string
wing: string
size: string
date: dictionary<values=string, indices=int32, ordered=0>
-- schema metadata --
metadata: '{'data_type': 2, 'title': 'Sales of Sanitary Napkins on Lazada' + 867
dictionary: '{'brand': 'ชื่อยี่ห้อผ้าอนา�' + 421

In [15]:
# Extracting metadata and data dictionary
metadata = eval(table.schema.metadata[b'metadata']) if b'metadata' in table.schema.metadata else None
data_dictionary = eval(table.schema.metadata[b'dictionary']) if b'dictionary' in table.schema.metadata else None

# Display the DataFrame, Metadata, and Data Dictionary
print("Metadata:", metadata)
print("Data Dictionary:", data_dictionary)

Metadata: {'data_type': 2, 'title': 'Sales of Sanitary Napkins on Lazada', 'owner_org': 'บริษัท ลาซาด้า จำกัด', 'maintainer': 'กลุ่มอาลีบาบา', 'maintainer_email': 'sevice@lazada.co.th', 'tag_string': 'brand, price, sale, total_sale, location, usage, length, type, wing, size', 'note': 'ยอดขายจากการจำหน่ายผ้าอนามัย,จำแนกตามรายชื่อยี่ห้อ,รูปแบบของผ้าอนามัย', 'objective': 99, 'update_frequency_unit': 'R', 'update_frequency_interval': 0, 'geo_coverage': '04', 'data_source': 'ข้อมูลการขายสินค้าและยอดขายในการให้บริการมีที่มาจากบริษัท ลาซาด้า จำกัด', 'data_format': 10, 'data_category': 1, 'license_id': 98}
Data Dictionary: {'brand': 'ชื่อยี่ห้อผ้าอนามัย', 'price': 'ราคา', 'sale': 'จำนวนที่ขาย', 'total_sale': 'ยอดขายรวม', 'location': 'จังหวัด', 'usage': 'การใช้งาน', 'length': 'ความยาว', 'type': 'รูปแบบ', 'wing': 'มีปีก,ไม่มีปีก', 'size': 'ขนาด', 'date': 'วันที่ได้การนำข้อมูลมา'}


In [16]:
# Convert to Pandas DataFrame
df = table.to_pandas()

# Display the DataFrame
print(df.head())

  brand  price    sale  total_sale location   usage  length type wing   size  \
0  Sofy  10.75    74.0       795.5  Bangkok   night    29.0  pad   no  thick   
1  Sofy  33.00  3269.0    107877.0  Bangkok     day    22.0  pad  yes   thin   
2  Sofy  11.00   748.0      8228.0  Bangkok   night    29.0  pad   no  thick   
3  Sofy  59.00   819.0     48321.0  Bangkok   night    29.0  pad  yes   thin   
4  Sofy  39.00   426.0     16614.0  Bangkok  normal    22.0  pad   no   thin   

         date  
0  2023-12-16  
1  2023-12-16  
2  2023-12-16  
3  2023-12-16  
4  2023-12-16  
